# Using CLIPSeg with Hugging Face Transformers

Using Hugging Face Transformers, you can easily download and run a pre-trained CLIPSeg model on your images. Let’s start by installing transformers.

To download the model, simply instantiate it.

In [ ]:
# import os
# HOME = os.getcwd()
# print("HOME:", HOME)

# %cd {HOME}
# !git clone https://github.com/IDEA-Research/GroundingDINO.git
# %cd {HOME}/GroundingDINO
# !git checkout -q 57535c5a79791cb76e36fdb64975271354f10251
# !pip install -q -e .
# !pip uninstall -y supervision
# !pip install -q supervision==0.6.0

# %cd {HOME}
# !mkdir -p {HOME}/weights
# %cd {HOME}/weights

# !wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

In [13]:
#path
repo_root = "../../"
sam_path = repo_root + "sam_vit_h_4b8939.pth"
Dataset_path = repo_root + "datasets/people_poses/"

In [2]:
import supervision as sv
print(sv.__version__)

import os

HOME = "/home/hongxin/"
GROUNDING_DINO_CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(GROUNDING_DINO_CONFIG_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CONFIG_PATH))


GROUNDING_DINO_CHECKPOINT_PATH = os.path.join(HOME, "weights", "groundingdino_swint_ogc.pth")
print(GROUNDING_DINO_CHECKPOINT_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CHECKPOINT_PATH))

import torch

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import sys
sys.path.append(os.path.join(HOME, "GroundingDINO"))

from groundingdino.util.inference import Model

grounding_dino_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)

0.6.0
/home/hongxin/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True
/home/hongxin/weights/groundingdino_swint_ogc.pth ; exist: True


/home/hongxin/anaconda3/envs/CSCI567/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hongxin/anaconda3/envs/CSCI567/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392035891/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


## Text prompting

Let’s start by defining some text categories we want to segment.

Now that we have our inputs, we can process them and input them to the model.

In [3]:
from typing import List

def enhance_class_name(class_names: List[str]) -> List[str]:
    return [
        f"all {class_name}s"
        for class_name
        in class_names
    ]

In [4]:
import torch
def getPredictions(promptValue, image):
  prompts = [promptValue]
  BOX_TRESHOLD = 0.35
  TEXT_TRESHOLD = 0.25
  pil_image = image.convert('RGB')
  open_cv_image = np.array(image)
  # Convert RGB to BGR
  open_cv_image = open_cv_image[:, :, ::-1].copy()

  detections = grounding_dino_model.predict_with_classes(
  image=open_cv_image,
  classes=enhance_class_name(class_names=prompts),
  box_threshold=BOX_TRESHOLD,
  text_threshold=TEXT_TRESHOLD
  )


  if len(detections.xyxy) > 0:
    return detections.xyxy[0]
  return [0, 0, 0, 0]

# New Section

In [5]:
from PIL import Image

def make_square_by_padding(img, fill_color=(0, 0, 0)):
    # img = Image.open(requests.get(url, stream=True).raw)
    width, height = img.size

    # Determine the size for the square
    new_size = max(width, height)

    # Create a new image with the desired size and fill color
    new_img = Image.new("RGB", (new_size, new_size), fill_color)

    # Paste the original image onto the center of the new image
    new_img.paste(img, ((new_size - width) // 2, (new_size - height) // 2))

    return new_img



In [6]:
import cv2
import matplotlib.pyplot as plt
from torchvision.ops import masks_to_boxes
import torchvision.transforms as T
import torchvision.transforms.functional as F

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image, Image, ImageDraw
import numpy as np
# ***
def processPredictionImage(pred, img):
    mask = torch.sigmoid(torch.reshape(torch.tensor(pred), (352, 352)))
    # ax[i+1].imshow(mask)

    processed_tensor = torch.sigmoid(torch.reshape(torch.tensor(pred), (352, 352)))
    image_np = (processed_tensor.detach().numpy() * 255).astype(np.uint8)

    _, binary_image = cv2.threshold(image_np, 127, 255, cv2.THRESH_BINARY)

    desired_width, desired_height = img.size
    resized_image = cv2.resize(binary_image, (max(desired_width, desired_height),  max(desired_width, desired_height)), interpolation=cv2.INTER_AREA)

    # Get dimensions of the binary image
    height, width = resized_image.shape[:2]

    # Check if the desired crop size is smaller than the original image size
    if desired_width <= width and desired_height <= height:
        # Calculate the top-left corner of the crop
        x = width // 2 - desired_width // 2
        y = height // 2 - desired_height // 2

        # Crop the image
        cropped_image = resized_image[y:y+desired_height, x:x+desired_width]
        return cropped_image

    else:
        print("failed to crop image")
        return resized_image

In [7]:
# ***
def getBoundingBox(predictionImage):
    contours, _ = cv2.findContours(predictionImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        list_of_pts = []
        for ctr in contours:
            list_of_pts += [pt[0] for pt in ctr]
        ctr = np.array(list_of_pts).reshape((-1,1,2)).astype(np.int32)
        # largest_contour = max(cv2.convexHull(ctr), key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cv2.convexHull(ctr))

        print(f"Bounding Box: x={x}, y={y}, width={(w)}, height={(h)}")
        return [x, y, (w + x), (h + y)]

    else:
        print("No contours found")
        return [0, 0, 0, 0]



# SAM Integration

In [ ]:
# !pip install torch opencv-python Pillow
# !pip install git+https://github.com/openai/CLIP.git
# !pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'
# !pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

# !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [8]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = sam_path
model_type = "vit_h"

# device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
# sam.to(device=device)

predictor = SamPredictor(sam)

In [9]:
def getSAMPreditction(image, box):

    image_np = np.array(image)
    image2 = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    predictor.set_image(image2)

    input_box = np.array(box)
    if(input_box[0] == 0 and input_box[1] == 0 and input_box[2] == 0 and input_box[3] == 0):
        masks, _, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            box=None,
            multimask_output=False,
        )
    else:
        masks, _, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            box=input_box[None, :],
            multimask_output=False,
        )
    return masks[0]

In [10]:
# ****
# numpy version
def pixAcc(predicted, target):
    same = (predicted == target).sum()
    w, h = target.shape
    print("Target seg shape: {}, Predicted seg shape: {}, #Same pixels: {}".format(target.shape, predicted.shape, same))
    return same / (w * h)

# input: bool matrix
def IOU(predicted , target):
    intersection = np.logical_and(target, predicted).sum()
    union = np.logical_or(target, predicted).sum()
    if union == 0:
        iou_score = 0
    else :
        iou_score = intersection / union
    return iou_score

In [11]:
# *****
def gt_to_anns_of_label_mask(mask_gt):
  labels = np.unique(mask_gt)
  anns = []
  for label in labels:
    # skip background
      if label == 0:
          continue
      mask = np.all(mask_gt == label, axis=-1)
      anns.append({
        'area': np.sum(mask),
        'segmentation': mask,
        'label': label,
      })
  return anns

# Final Results

In [12]:
import numpy as np
# ***

def getFinalPredictions(images, promptVals):
  preds = []
  boundingBoxes = []
  masks = []
  emptyPrompt = ""
  for img in images:
    imgResults = {}
    imageBoundingBoxes = {}
    imageMasks = {}
    img_with_border = make_square_by_padding(img)

    for prompt in promptVals:
      # prediction = getPredictions(prompt, img)
  #     imgResults[prompt] = prediction
  #     predictionImage = processPredictionImage(prediction, img)

      bbox = getPredictions(prompt, img)
      imageBoundingBoxes[prompt] = bbox
      print(bbox)
      if(bbox[0] == 0 and bbox[1] == 0 and bbox[2] == 0 and bbox[3] == 0 and emptyPrompt == ""):
        print("Here")
        mask = getSAMPreditction(img, bbox)
        imageMasks[prompt] = mask
        emptyPrompt = prompt
      elif(bbox[0] == 0 and bbox[1] == 0 and bbox[2] == 0 and bbox[3] == 0):
        print("Here")
        imageMasks[prompt] = imageMasks[emptyPrompt]
      else:
        mask = getSAMPreditction(img, bbox)
        imageMasks[prompt] = mask

    emptyPrompt = ""
    preds.append(imgResults)
    boundingBoxes.append(imageBoundingBoxes)
    masks.append(imageMasks)

  preds = np.array(preds)
  boundingBoxes = np.array(boundingBoxes)
  return boundingBoxes, preds, masks


In [14]:
import os
from tqdm import tqdm
import copy
# ***
def getImages(num):
    images = []
    truthMasks = []
    data_list = []
    root = ""
    root = Dataset_path
    textFile = "val_id.short.txt"
    imageFile = "val_images/"
    segmentationFile = "val_segmentations/"
    with open(os.path.join(root, f"val_id.short.txt"), 'r') as lf:
        data_list = [ s.strip() for s in lf.readlines() ]

    num_valid_case, sum_miou, sum_pixAcc = 0,0,0
    try:
        for data_name in (pbar := tqdm(data_list[:num])):
            img_path = root + imageFile + data_name + '.jpg'
            seg_path = root +  segmentationFile+ data_name + '.png'

            # Read Image and Ground truth mask
            img = copy.deepcopy(Image.open(img_path))

            # display(img)
            if img is None:
                print("\nimage is None", data_name)
                continue
            else:
                images.append(img)
            mask_gt = cv2.imread(seg_path)
            if mask_gt is None:
                print("\nmask_gt is None", data_name)
                continue
            else:
                truthMasks.append(mask_gt)
    except Exception as e:
        print("ERROR")
        print(e)
    return images, truthMasks


In [15]:
def get_masked_image(original_image, segmentation):
  # Visualize

  overlay_image = Image.new('RGBA', original_image.size, (0, 0, 0, 0))
  overlay_color = (255, 0, 0, 200)

  draw = ImageDraw.Draw(overlay_image)
  segmentation_mask_image = Image.fromarray(segmentation.astype('uint8') * 255)
  draw.bitmap((0, 0), segmentation_mask_image, fill=overlay_color)

  return Image.alpha_composite(original_image.convert('RGBA'), overlay_image)

In [16]:

def evaluate(images, masks, truthMasks):
  # Get confidence scores for each masks generated by SAM for
  # each object label existing in the given image

  results = []
  label_name = ["Background","Hat","Hair","Glove",
        "Sunglasses","UpperClothes","Dress","Coat","Socks","Pants",
        "Jumpsuits","Scarf","Skirt","Face","Left-arm","Right-arm","Left-leg","Right-leg","Left-shoe","Right-shoe"]


  for i in range(len(truthMasks)):
    anns = gt_to_anns_of_label_mask(truthMasks[i])
    for ann in anns:
      mask = masks[i]
      image = images[i]
      label =  label_name[ann['label']]
      print(label_name[ann['label']])
      # Get Corresponding gt Mask, generated Mask for Evaluation
      print(mask[label])
      iou = IOU(mask[label],ann['segmentation'])
      print(iou)
      result_image = get_masked_image(image, mask[label])
      # display(result_image)
      pixacc = pixAcc(mask[label],ann['segmentation'])
      result = {
          "iou": iou,
          "pixacc": pixacc,
          "label_num": ann["label"],
          "label_name": label,
          "masked_img": result_image,
          "pred_mask": mask[label],
          "gt_mask": ann
      }
      print("iou:{}, pixacc:{}, label num:{}, label_name:{}".format(iou, pixacc, ann["label"], label))
      results.append(result)

  return results

# New Section

In [17]:
import numpy as np
# ***
# TODO replace for body pose images
images, truthMasks = getImages(32)

bodyPrompts = ["Background","Hat","Hair","Glove", "Sunglasses","UpperClothes","Dress","Coat","Socks","Pants", "Jumpsuits","Scarf","Skirt","Face","Left-arm","Right-arm","Left-leg","Right-leg","Left-shoe","Right-shoe"]
# bodyPrompts = ["Socks", "Hat"]

boundingBoxes, preds, masks = getFinalPredictions(images, bodyPrompts)



100%|██████████| 20/20 [00:00<00:00, 374.13it/s]
/home/hongxin/anaconda3/envs/CSCI567/lib/python3.11/site-packages/transformers/modeling_utils.py:907: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/hongxin/anaconda3/envs/CSCI567/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/hongxin/anaconda3/envs/CSCI567/lib/python3.11/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[  1.7421188   1.2724304 185.65591   420.99112  ]
[ 3.8595352   0.20902061 90.265945   63.996582  ]
[0, 0, 0, 0]
Here
[0, 0, 0, 0]
Here
[0, 0, 0, 0]
Here
[0, 0, 0, 0]
Here
[ 52.9701   70.34105 185.13232 256.18842]
[ 53.607075  71.27692  159.0787   203.17712 ]
[ 87.97363 302.73065 162.02223 395.635  ]
[ 86.43402 175.08844 184.979   255.94565]
[  5.7838593   1.3628998 184.51509   421.50684  ]
[0, 0, 0, 0]
Here
[ 86.60849 174.96065 185.04904 256.03882]
[  6.435463    1.3731537 184.4891    421.16614  ]
[ 45.861942  86.58783  131.6577   247.71976 ]
[ 45.85623   82.426414 131.74103  247.75131 ]
[ 61.18635 246.30984 176.63368 421.71414]
[ 60.9741  246.16763 176.59479 421.91232]
[ 60.29999 388.6981  136.29457 422.93027]
[ 60.28065 388.6683  136.27907 422.9415 ]
[  0.9369507    0.90563965 139.2711     317.09702   ]
[ 7.208317    0.71469116 87.03218    64.61394   ]
[ 7.4671135  1.0206795 86.77367   66.64192  ]
[ 59.190617 239.35081  107.584    297.4223  ]
[0, 0, 0, 0]
Here
[  5.8107834  74.26195

KeyboardInterrupt: 

# New Section

In [ ]:
def compute_pix_acc(predicted, target):

    assert predicted.shape == target.shape
    assert len(predicted.shape) == 2
    return (predicted == target).mean()

def compute_IOU(predicted, target):

    assert predicted.shape == target.shape

    assert len(predicted.shape) == 2
    intersection = np.logical_and(target, predicted).sum()
    union = np.logical_or(target, predicted).sum()
    assert union > 0
    return intersection / union
def compute_metric(name, masks, label):
    """ name: data_id
        mask: { label_id: numpy.ndarray(shape=(H, W)) }
        label: np.ndarray(shape=(H, W)) --> numbers from 0 to 19
    """
    pix_acc_metric = { "name": name }
    iou_metric = { "name": name }
    empty = np.zeros_like(label)
    LABELS = ["Background","Hat","Hair","Glove",
        "Sunglasses","UpperClothes","Dress","Coat","Socks","Pants",
        "Jumpsuits","Scarf","Skirt","Face","Left-arm","Right-arm","Left-leg","Right-leg","Left-shoe","Right-shoe"]

    for i, label_name in enumerate(LABELS):
        mask_i = masks.get(label_name, empty)
        label_i = (label == i)
        if label_i.sum() == 0:
            # pandas dataframe automatically skips nan
            # when computing .count() and .mean()
            iou_metric[label_name] = np.nan
            pix_acc_metric[label_name] = np.nan
        else:
            iou_metric[label_name] = compute_IOU(mask_i, label_i)
            pix_acc_metric[label_name] = compute_pix_acc(mask_i, label_i)

    return iou_metric, pix_acc_metric

In [ ]:
def getImages2(num):
    images = []
    truthMasks = []
    data_list2 = []
    root = ""
    root = "/content/drive/MyDrive/CSCI 567 /segment-anything/datasets/people_poses/"
    textFile = "val_id.short.txt"
    imageFile = "val_images/"
    segmentationFile = "val_segmentations/"
    with open(os.path.join(root, f"val_id.short.txt"), 'r') as lf:
        data_list = [ s.strip() for s in lf.readlines() ]

    num_valid_case, sum_miou, sum_pixAcc = 0,0,0
    try:
        for data_name in (pbar := tqdm(data_list[:num])):
            img_path = root + imageFile + data_name + '.jpg'
            seg_path = root +  segmentationFile+ data_name + '.png'

            # Read Image and Ground truth mask
            img = copy.deepcopy(Image.open(img_path))

            # display(img)
            if img is None:
                print("\nimage is None", data_name)
                continue
            else:
                data_list2.append(data_name)
                images.append(img)
            mask_gt = cv2.imread(seg_path)
            if mask_gt is None:
                print("\nmask_gt is None", data_name)
                continue
            else:
                truthMasks.append(mask_gt)
    except Exception as e:
        print("ERROR")
        print(e)
    return images, truthMasks, data_list2


In [ ]:
# import pandas as pd
# results = evaluate(images, masks, truthMasks)
from PIL import Image, Image, ImageDraw
from torch.jit import Error
import pandas as pd
import copy
# numpy metrics
import os
from tqdm import tqdm
# root = "/content/drive/MyDrive/CSCI567/segment-anything/datasets/people_poses/"
root = ""
prompt = "The object of "

# try:
miou_table = []
pix_acc_table = []
images, truthMasks, data_list = getImages2(32)
print(len(truthMasks))
print(len(images))
print(len(data_list))

result_table = {}
i = 0
try:
  for data_name in data_list:

    anns = gt_to_anns_of_label_mask(truthMasks[i])
    miou, pix_acc = compute_metric(data_name, masks[i], truthMasks[i][:,:,0])
    miou_table.append(miou)
    pix_acc_table.append(pix_acc)
    predict_anns = []
    for ann in anns:

      ## Get Label Index with Highest Score
      predict_anns.append({
          'segmentation': masks[i],
          'label': ann['label'],
          'gt': ann['segmentation'],
      })
    i += 1
    result_table[data_name] = predict_anns


except Exception as e:
  print(e)
  print(miou_table)
  print(pix_acc_table)

miou_table = pd.DataFrame(miou_table, columns=miou_table[0].keys()).set_index('name')
# miou_table.to_csv('/Users/arianasokolov/Desktop/ClipSegSam/CSVs/miou.csv')
# np.save('vis_cliseg_sam_32.npy', result_table)

pix_acc_table = pd.DataFrame(pix_acc_table, columns=pix_acc_table[0].keys()).set_index('name')
# pix_acc_table.to_csv('/Users/arianasokolov/Desktop/ClipSegSam/CSVs/pix_acc.csv')

# New Section

Finally, let’s visualize the output.

In [ ]:
print(result_table)

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
def get_mask(mask,color=None):
    if color is None:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    # else:
    #     color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    return mask_image

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    return plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2)

In [ ]:
prompts = bodyPrompts[:3]
last = 2
# *****
# Setup plot
_, ax = plt.subplots(len(boundingBoxes[:last]), len(prompts) + 1, figsize=(3*(len(prompts) + 1), 4))
[a.axis('off') for a in ax.flatten()]

for i in range(len(boundingBoxes[:last])):
  # Show padded image
  ax[0].imshow(images[i])

  for j in range(len(prompts)):
    # Show mask
    print(preds[i])
    # predictionImage = processPredictionImage(preds[i][prompts[j]], images[i])
    # ax[i][j+1].imshow(predictionImage)
    # ax[i][j+1].imshow(get_mask(masks[0]["Socks"]))

    # Show bounding box
    print(boundingBoxes[i][prompts[j]])
    ax[0].add_patch(show_box(boundingBoxes[i][prompts[j]], plt.gca()))


    # Show prompt
    # ax[i][j+1].text(0, -15, prompts[j])

In [ ]:
prompts = ["Right-arm", "Left-arm", "Right-leg", "Left-leg", "Hat", "Socks"]
# Setup plot
_, ax = plt.subplots(len(boundingBoxes[:last]), len(prompts) + 1, figsize=(3*(len(prompts) + 1), 4))
[a.axis('off') for a in ax.flatten()]

for i in range(len(prompts)):
  # Show padded image\
  ax[0][0].text( 0, 0,"orig")
  ax[0][0].imshow(images[0])
  print(prompts[i])
  ax[0][i + 1].text( 0, 0, prompts[i])
  ax[0][i + 1 ].imshow(get_mask(masks[0][prompts[i]], color=colors[i]))
  ax[1][i + 1].text( 0, 0, prompts[i])
  ax[1][i + 1 ].imshow(get_mask(truthMasks[0][i], color=colors[i]))
  # for j in range(len(prompts)):

  #   # Show mask
  #   print(preds[i])
  #   predictionImage = processPredictionImage(preds[i][prompts[j]], images[i])
  #   ax[i][j+1].imshow(predictionImage)

    # ax[i][j+1].imshow(images[i])
    # ax[i][j+1].imshow(get_mask(masks[i]["Socks"], color=colors[j]))

    # Show bounding box
    # print(boundingBoxes[i][prompts[j]])
    # ax[i][j+1].add_patch(show_box(boundingBoxes[i][prompts[j]], plt.gca()))


    # Show prompt
    # ax[i][j+1].text(0, -15, prompts[j])

In [ ]:
input_box = np.array(boundingBoxes[0]["Socks"])


plt.figure(figsize=(10, 10))
plt.imshow(images[0])
show_mask(masks[0]["Socks"], plt.gca())
# plt.gca().add_patch(show_box(input_box, plt.gca()))
plt.axis('off')
plt.show()